Welcome to the PyFlowline tutorial notebook!
This is a tutorial pyflowline notebook.

This tutorial serves as an example of the PyFlowline application using a dggrid mesh.

For additional information on this application and the DGGRID mesh, please refer to the following publication:

Liao, C., Engwirda, D., Cooper, M., Li, M., and Fang, Y.: Discrete Global Grid System-based Flow Routing Datasets in the Amazon and Yukon Basins, Earth Syst. Sci. Data Discuss. [preprint], https://doi.org/10.5194/essd-2023-398, in review, 2024.

If you are running this notebook directly from the Binder platform, then all the dependencies are already installed. Otherwise, you must install the PyFlowline package and its dependencies. Additionally, visualization requires optional dependency packages (refer to the full documentation installation section).

Feel free to modify the notebook to use a different visualization method as needed. Enjoy exploring PyFlowline!


First, let's load some Python libraries.

In [ ]:
import os
import json
import shutil
from pathlib import Path
from os.path import realpath
import importlib.util
from shutil import copy2
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt

Check pyflowline installation.

In [ ]:
if importlib.util.find_spec("pyflowline") is not None:
    print('The pyflowline package is installed. ')
else:
    print('The pyflowline package is not installed. Please install it first.')
    exit()

Add dggrid into the system path.

In [ ]:
os.environ["PATH"] += os.pathsep + "/home/jovyan/"

Now we can import a few python function within pyflowline.

In [ ]:
#load the read configuration function
from pyflowline.configuration.change_json_key_value import change_json_key_value
from pyflowline.configuration.read_configuration_file import pyflowline_read_configuration_file

Preperae the input/output data structure.

In [ ]:
sPath_notebook = Path().resolve()
sPath_parent = str(Path().resolve().parents[1])
print(sPath_parent)

sWorkspace_data = os.path.join( sPath_parent, 'data', 'yukon' )
if not os.path.exists(sWorkspace_data):
    print(sWorkspace_data)
    os.makedirs(sWorkspace_data)

sWorkspace_input = os.path.join( sWorkspace_data, 'input')
if not os.path.exists(sWorkspace_input):
    print(sWorkspace_input)
    os.makedirs(sWorkspace_input)

sWorkspace_output = os.path.join( sWorkspace_data, 'output')
if not os.path.exists(sWorkspace_output):
    print(sWorkspace_output)
    os.makedirs(sWorkspace_output)

Create a temp folder to download data.

In [ ]:
sPath_temp = os.path.join( sPath_parent, 'data', 'tmp' )
if not os.path.exists(sPath_temp):
    print(sPath_temp)
    os.makedirs(sPath_temp)
else:
    shutil.rmtree(sPath_temp)

# Specify the repository's URL
hexwatershed_data_repo = 'https://github.com/changliao1025/hexwatershed_data.git'

# Clone the repository
os.system(f'git clone {hexwatershed_data_repo} {sPath_temp}')
sPath_temp_data = os.path.join(sPath_parent, 'data', 'tmp', 'data', 'yukon', 'input')

# Check if the destination directory exists, if exists, remove it
if os.path.exists(sWorkspace_input):
    shutil.rmtree(sWorkspace_input)

# Copy all the files under the temp data folder using shutil
shutil.copytree(sPath_temp_data, sWorkspace_input)

pyflowline uses a json file for configuration, an example json file is provided.
check whether a configuration exists.

In [ ]:
sFilename_configuration_in = realpath(os.path.join(sWorkspace_input , 'pyhexwatershed_yukon_dggrid.json'))
sFilename_basins_in = realpath( os.path.join( sWorkspace_input , 'pyflowline_yukon_basins.json') )
if os.path.isfile(sFilename_configuration_in):
    pass
else:
    print('The domain configuration file does not exist: ', sFilename_configuration_in)

print('Finished the data preparation step.')

Check the contents of the json configuration file.

In [ ]:
with open(sFilename_configuration_in, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed, indent=4))

The meaning of these json keywords are explained in the [pyflowline documentation](https://pyflowline.readthedocs.io/en/latest/data/data.html#inputs).

Now set up some keywords.

In [ ]:
# Set some parameters.
sRegion = 'yukon'
sMesh_type = 'dggrid'
sDggrid_type = 'ISEA3H'
iCase_index = 1
iResolution_index = 10 # dggrid resolution index

# Get today's year, month and day.
today = date.today()
iYear = today.year
iMonth = today.month
iDay = today.day
print("Today's date:", iYear, iMonth, iDay)
sDate = str(iYear) + str(iMonth).zfill(2) + str(iDay).zfill(2)

In [ ]:
from pyflowline.mesh.dggrid.create_dggrid_mesh import dggrid_find_resolution_by_index
dResolution = dggrid_find_resolution_by_index(sDggrid_type,
                                              iResolution_index)
print(dResolution)

The pyflowline python package uses the OOP approach to manage each simulation. A pyflowline object is created by reading the configuration file.

The first argument to the `pyflowline_read_configuration_file` function is the configuration file filename, followed by name-value keywords that correspond to the parameters in the json configuration files.

In [ ]:
#instead of changing the main configuration file directly, we will make copies
# Copy the configuration file to the output directory.
sFilename_configuration_copy = os.path.join(
    sWorkspace_output, 'pyflowline_configuration_copy.json')
copy2(sFilename_configuration_in, sFilename_configuration_copy)

# Also copy the basin configuration file to the output directory.
sFilename_basins_configuration_copy = os.path.join(
    sWorkspace_output, 'pyflowline_configuration_basins_copy.json')
copy2(sFilename_basins_in, sFilename_basins_configuration_copy)

Update a few parameters in the configuration file before we can create the flowline object.

In [ ]:
# The json file will be overwritten, you may want to make a copy of it first.
sFilename_configuration = sFilename_configuration_copy
sFilename_basins = sFilename_basins_configuration_copy
# Output folder
change_json_key_value(sFilename_configuration,
                                 'sWorkspace_output', sWorkspace_output)

# Individual basin configuration file
change_json_key_value(sFilename_configuration,
                                 'sFilename_basins', sFilename_basins)

# Boundary to clip mesh
sFilename_mesh_boundary = realpath(os.path.join(
    sWorkspace_input, 'boundary.geojson'))
change_json_key_value(sFilename_configuration,
                                 'sFilename_mesh_boundary', sFilename_mesh_boundary)

We can now call the function to create an object.

In [ ]:
#the read function accepts several keyword arguments that can be used to change the default parameters.
oPyflowline = pyflowline_read_configuration_file(sFilename_configuration,
                                                 iCase_index_in=iCase_index,
                                                 sMesh_type_in=sMesh_type,
                                                 iResolution_index_in=iResolution_index,
                                                 sDate_in=sDate)

You can review the setting again.

In [ ]:
print(oPyflowline.tojson())

If you are not certain of the outlet location, you can also set them up using:

In [ ]:
# Another important setting for basin is the approximate outlet location
# You can set it using the change_model_parameter function.
dLongitude_outlet_degree = -164.47594
dLatitude_outlet_degree = 63.04269
oPyflowline.aBasin[0].dThreshold_small_river = dResolution * 5

oPyflowline.pyflowline_change_model_parameter('dLongitude_outlet_degree',
                                              dLongitude_outlet_degree,
                                              iFlag_basin_in=1)

oPyflowline.pyflowline_change_model_parameter('dLatitude_outlet_degree',
                                              dLatitude_outlet_degree,
                                              iFlag_basin_in=1)
sFilename_flowline = realpath(os.path.join(sWorkspace_input, 'dggrid10/river_networks.geojson') )
oPyflowline.pyflowline_change_model_parameter('sFilename_flowline_filter', sFilename_flowline, iFlag_basin_in= 1)
oPyflowline.pyflowline_change_model_parameter('iFlag_debug', 0, iFlag_basin_in= 1)


You can check the setting for the single basin as well

In [ ]:
print(oPyflowline.aBasin[0].tojson())

After the case object was created, we can set up the model.

In [ ]:
#setup the model
oPyflowline.iFlag_user_provided_binary = 0
oPyflowline.pyflowline_setup()

Before any operation, we can visualize the original or raw flowline dataset. 

In [ ]:
sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_filter_geojson
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()

You can also use QGIS.

The plot function provides a few optional arguments such as map projection and spatial extent. 
By default, the spatial extent is full. 
But you can set the extent to a zoom-in region.

Now let's run the three major steps/operations in the pyflowline algorithm one by one.

In [ ]:
# Run step 1: flowline simplification.
oPyflowline.pyflowline_flowline_simplification();

Check the result using a plot.

In [ ]:
sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_simplified
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()

Similarly, we can zoom in using the extent.

Next, we will creata a mesh from the global MPAS mesh.

In [ ]:
# Run step 2: create a mesh.
# We can either use a rectangle boundary
oPyflowline.iFlag_mesh_boundary = 1
aCell = oPyflowline.pyflowline_mesh_generation()

In [ ]:
# Visualize the mesh
sFilename_geojson = oPyflowline.sFilename_mesh
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()

we can also use a polygon to create a mesh

Last, we can generate the conceptual flowline. We refer to the final flowline as "conceptual" because it has been modified relative to the original, input flowline, which often represents a "real" flowline. The conceptual flowline has been simplified (e.g., small reaches, loops, and braided channels removed) and adjusted to align with the mesh. These modifications ensure the final flowline is suitable for hydrological modeling, while remaining consistent with the real flowline.

In [ ]:
# Run Step 3: create the "conceptual" (topological) flowline.
oPyflowline.pyflowline_reconstruct_topological_relationship();

Now we can overlap mesh with flowline.

In [ ]:
# Plot both the mesh and the flowline
sFilename_mesh = oPyflowline.sFilename_mesh
sFilename_conceptual_flowline = oPyflowline.aBasin[0].sFilename_flowline_conceptual
gdf1 = gpd.read_file(sFilename_mesh)
gdf2 = gpd.read_file(sFilename_conceptual_flowline)
fig, ax = plt.subplots()
gdf1.plot(ax=ax, color='blue')
gdf2.plot(ax=ax, color='red')
plt.show()

After this, we can save the model output into a json file.

In [ ]:
# Export output
oPyflowline.pyflowline_export();

The content of the one of the exported json files can be checked:

In [ ]:
with open(oPyflowline.sFilename_mesh_info, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed[0], indent=4))

Congratulations! You have successfully finished a pyflowline simulation.